In [55]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import pydst
dst = pydst.Dst(lang='en')

In [56]:
var_dict = {} # var is for variable
var_dict['EJENDOMSKATE'] = 'Y'

In [68]:
# a. load
ej14_true = dst.get_data(table_id = 'ej14', variables={'REGION':['*'], 'EJENDOMSKATE':['*'], 'TAL':['*'], 'TID':['*']})



I = ej14_true.EJENDOMSKATE.str.contains('Owner')
I |= ej14_true.TAL.str.contains('Percentage')
ej14_true.loc[I, :]

ej14_true = ej14_true.loc[I == False] # keep everything else
ej14_true.head(10)

,REGION,EJENDOMSKATE,TAL,TID,INDHOLD
0,All Denmark,One-family houses,Index,2006M01,92.5
6,Region Nordjylland,One-family houses,Index,2006M01,94.1
12,Region Midtjylland,One-family houses,Index,2006M01,93.9
18,Region Syddanmark,One-family houses,Index,2006M01,94.6
24,Region Hovedstaden,One-family houses,Index,2006M01,90.0
30,Region Sjælland,One-family houses,Index,2006M01,91.7
36,All Denmark,One-family houses,Index,2006M02,94.2
42,Region Nordjylland,One-family houses,Index,2006M02,97.4
48,Region Midtjylland,One-family houses,Index,2006M02,94.6
54,Region Syddanmark,One-family houses,Index,2006M02,94.2


In [72]:
ej14_true = ej14_true.sort_values(['REGION', 'TID'])
ej14_true.head(5)

,REGION,EJENDOMSKATE,TAL,TID,INDHOLD
0,All Denmark,One-family houses,Index,2006M01,92.5
36,All Denmark,One-family houses,Index,2006M02,94.2
72,All Denmark,One-family houses,Index,2006M03,96.4
108,All Denmark,One-family houses,Index,2006M04,98.8
144,All Denmark,One-family houses,Index,2006M05,100.4


In [73]:
rate = dst.get_data(table_id = 'DNRENTM', variables={'INSTRUMENT':['*'], 'LAND':['*'], 'OPGOER':['*'], 'TID':['*']})

I = rate.TID.str.contains('1985')
I |= rate.TID.str.contains('1986')
I |= rate.TID.str.contains('1987')
I |= rate.TID.str.contains('1988')
I |= rate.TID.str.contains('1989')
I |= rate.TID.str.contains('1990')
I |= rate.TID.str.contains('1991')
I |= rate.TID.str.contains('1992')
I |= rate.TID.str.contains('1993')
I |= rate.TID.str.contains('1994')
I |= rate.TID.str.contains('1995')
I |= rate.TID.str.contains('1996')
I |= rate.TID.str.contains('1997')
I |= rate.TID.str.contains('1998')
I |= rate.TID.str.contains('1999')
I |= rate.TID.str.contains('2000')
I |= rate.TID.str.contains('2001')
I |= rate.TID.str.contains('2002')
I |= rate.TID.str.contains('2003')
I |= rate.TID.str.contains('2004')
I |= rate.TID.str.contains('2005')
rate.loc[I, :]

rate = rate.loc[I == False] # keep everything else

I = rate.OPGOER.str.contains('Monthly')
rate.loc[I, :]

rate = rate.loc[I == False] # keep everything else

I = rate.INSTRUMENT.str.contains('Certificates')
rate.loc[I, :]

rate = rate.loc[I == True] # keep everything else

del rate['LAND']
del rate['OPGOER']
del rate['INSTRUMENT']

rate = rate.sort_values(['TID'])

rate.rename(columns = {'INDHOLD':'Rate_value'}, inplace=True)


rate.head(10)

,TID,Rate_value
6640,2006M01,2.4000
6641,2006M02,2.5000
30432,2006M03,2.7500
30433,2006M04,2.7500
30616,2006M05,2.7500
30617,2006M06,3.0000
20854,2006M07,3.0000
20855,2006M08,3.2500
21038,2006M09,3.2500
21039,2006M10,3.5000


In [80]:
merged_true = pd.merge(ej14_true,rate,how='inner',on=['TID'])


merged_true.rename(columns ={'INDHOLD':'House price index (2006=100)'}, inplace=True)

del merged_true['EJENDOMSKATE']
del merged_true['TAL']

merged_true = merged_true.sort_values(['REGION', 'TID'])

merged_true.head(10)

,REGION,TID,House price index (2006=100),Rate_value
0,All Denmark,2006M01,92.5,2.4000
6,All Denmark,2006M02,94.2,2.5000
12,All Denmark,2006M03,96.4,2.7500
18,All Denmark,2006M04,98.8,2.7500
24,All Denmark,2006M05,100.4,2.7500
30,All Denmark,2006M06,101.9,3.0000
36,All Denmark,2006M07,103.0,3.0000
42,All Denmark,2006M08,103.4,3.2500
48,All Denmark,2006M09,102.1,3.2500
54,All Denmark,2006M10,102.7,3.5000


In [ ]:
# d. plot
def plot(merged_true):
    df_indexed = df.set_index('year')
    df_indexed[I].groupby(['variable'])['indexed'].plot(legend=True);